### Machine Learning - Genero Porthidium Colombia only females (July 2020)
### Morphotax & Ecology data

In [ ]:
import pandas as pd
import numpy as np
import scipy, os, sys
import collections

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

from IPython.display import HTML, display

import plotly.express as px
import plotly.graph_objs as go

sys.path.insert(1, '../../src/')
from Basic import *
import snake_lib as slib
from stat_lib import *

import statsmodels.api as sm
from statsmodels.stats.multicomp import MultiComparison  # Tukey
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec  # need for Tukey plot

import seaborn as sns
%matplotlib inline

#  Columns
- Voucher - id collection
- RDS - HEMC - half body number scales
- VS  - Ventral scales
- StS - subTail scales
- SRS - supra-right scales
- SLS - supra-left scales
- IRS - infra-right scales
- ILS - infra-left scales
- TL  - total length
- HL  - head length
- TaL - tail length

In [ ]:
all_fields = ['species', 'ecopop', 'voucher', 'gender', 'RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS', 'TL', 'HL', 'TaL', 'temperature', 'elevation']
all_fields2 = ['species', 'ecopop', 'voucher', 'gender', 'gender2', 'RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS', 'TL', 'HL', 'TaL', 'temperature', 'elevation']
numfields = ['RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS', 'TL', 'HL', 'TaL', 'temperature', 'elevation']
typefields = ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'C', 'C', 'C', 'C', 'C']

floatfields = ['HL', 'TaL', 'TL', 'temperature', 'elevation']
intfields   = ['RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS']

In [ ]:
root0     = "../../../colaboracoes/sergio_cubides_cubillos/"
root_data = os.path.join(root0, 'data2020/')
root_res  = os.path.join(root0, 'results/')

root_html   = os.path.join(root_res, 'html/')
root_figure = os.path.join(root_res, 'figures/')
root_maps   = os.path.join(root_res, 'maps/')
root_table  = os.path.join(root_res, 'tables/')


In [ ]:
filedata = "Porthidium Morphotaxonomy_20200530.csv"
filefull = os.path.join(root_data, filedata)
prefix = "Porthidium Morphotaxonomy"
sufix = 'porthidium_morphotaxonomy'

data = pdreadcsv(filedata, root_data)
print(data.columns)

all_fields = ['species', 'ecopop', 'voucher', 'sex', 
              'RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS', 'TL', 'HL', 'TaL', 'temperature', 'elevation']

data.columns = all_fields

data.species = data.species.str.strip()
data.ecopop  = data.ecopop.str.strip()
data.sex  = data.sex.str.strip().str.lower()

data["gender"] = [0 if sex == 'female' else 1 for sex in data.sex]
data.head()

In [ ]:
data.isnull().any()

In [ ]:
data.columns[data.isnull().any()]

In [ ]:
pd.isnull(data).any(axis=0)

In [ ]:
pd.isnull(data).any(axis=1)

In [ ]:
data[pd.isnull(data).any(axis=1)]

### Filter gender, how many?

In [ ]:
ngender = 0

if ngender == 0:
    sufix_gender = "female"
else:
    sufix_gender = "male"
    
    
df = data.copy()
df = df[df.gender == ngender]

nanimals = data.shape[0]
nmales   = np.sum(data.sex == 'male')
nfemales = np.sum(data.sex == 'female')

print("From %d snakes, there are %d males, and %d females in data."%(nanimals, nmales, nfemales))
print("There are %d %s in df."%(df.shape[0], sufix_gender))
df.head()

In [ ]:
df[df.gender == 1]  # no males

### Group by

### data == all data

In [ ]:
data.groupby('species').count().voucher

In [ ]:
data.groupby('ecopop').count().voucher

In [ ]:
dg = data.groupby(['ecopop', 'species', 'gender']).count().voucher
dg

In [ ]:
pdwritecsv(dg, 'groups.tsv', root_table, index=True)

In [ ]:
df.groupby('species').count().voucher

### df = determined gender

In [ ]:
df.groupby('species').count().voucher

In [ ]:
df.groupby('ecopop').count().voucher

### Sample SD are differents!

In [ ]:
pd.concat([df.groupby('species').temperature.mean().round(1), df.groupby('species').temperature.std().round(1)], axis=1, sort=False)

In [ ]:
df.groupby('species').temperature.agg(['mean', 'std', 'count']).round(2)

In [ ]:
specList = data.species.unique()
popList  = data.ecopop.unique()

In [ ]:
specList

In [ ]:
popList

In [ ]:
dfstat_specpop = slib.calc_snake_params_per_species_pop(data, numfields, typefields, sufix, root_table, force=False)
dfstat_specpop.head(3)

In [ ]:
dfstat_spec = slib.calc_snake_params_per_species(data, numfields, typefields, sufix, root_table, force=False)
dfstat_spec.head(3)

### Inter species-population

In [ ]:
dfisp = slib.htest_inter_species_population(data, numfields, typefields, root_table, force=False)
print(dfisp.shape)
dfisp.head(3)

#### Which are the most significat inter species-population

In [ ]:
dfisp2 = dfisp[dfisp.stat_significant == True]
dfisp2.head(3)

In [ ]:
dfisp2.measure.unique()

### Tukey test

In [ ]:
from statsmodels.stats.multicomp import MultiComparison
import matplotlib.gridspec as gridspec

units = ["cm", "cm",  "cm", "°C", "m"]
print(floatfields)
print(units)
sufix_gender

In [ ]:
df.head(2)

In [ ]:
# https://www.statsmodels.org/stable/generated/statsmodels.stats.multicomp.pairwise_tukeyhsd.html
# https://www.statsmodels.org/stable/generated/statsmodels.sandbox.stats.multicomp.TukeyHSDResults.html#statsmodels.sandbox.stats.multicomp.TukeyHSDResults
numfig = 0; verbose = False; dpi=300; force=False

for line in range(len(floatfields)):
    var = floatfields[numfig]
    unit = units[numfig]
    numfig += 1
    title = "%s for %s"%(var, sufix_gender)
    fig = plt.figure(figsize=(18,80), dpi=300, constrained_layout=False);
    
    dfa = data[ ['species', 'gender', 'sex', var]].copy()
    dfa['group'] = dfa.species + '-' + dfa.sex

    model = ols('%s ~ group'%(var), data=dfa).fit()
    aov_table = sm.stats.anova_lm(model, typ=2) # typ=2  # Type 2 Anova DataFrame

    cardata = MultiComparison(dfa[var], dfa.group)
    results = cardata.tukeyhsd()
    # print(results.summary())
    dfa = results_summary_to_dataframe(results)
    dfa = dfa.sort_values("pvalue_adj", ascending=True)
    filestatresult = "tukey_stat_between_species_and_gender_for_'%s'.tsv"%(var)
    if not os.path.exists(os.path.join(root_table, filestatresult)) or force:
        ret = pdwritecsv(dfa, filestatresult, root_table)
    
    if verbose:
        print(var, "ANOVA")
        print(aov_table)
        print("")
        print("MultiComparison results")
        print(dfa[dfa.pvalue_adj < 0.05])
    
    title  = "Species + Gender ~ '%s' - Tukey test: multiple comparisons between all pairs"%(var)
    xlabel = "%s (%s)"%(var,unit)
    
    results.plot_simultaneous();
    plt.title(title);
    plt.xlabel(xlabel);
    
    filefig = filestatresult.replace(".tsv", ".png")
    fname = os.path.join(root_figure, filefig)
    if not os.path.exists(fname) or force:
        plt.savefig(fname, dpi=dpi, facecolor='w', edgecolor='w', orientation='landscape')
        print("Figure saved at '%s'"%(fname))
    
    if verbose: print("") 
    
# plt.subplots_adjust(left=0.10, right=0.95, bottom=0.05, top=0.95, hspace=1.2) 

### Inter-species

In [ ]:
dfis = slib.htest_inter_species(data, numfields, typefields, root_table, force=False)
print(dfis.shape)
dfis.head(3)

In [ ]:
dfis2 = dfis[dfis.stat_significant == True]
dfis2.head(3)

In [ ]:
dfis2.measure.unique()

### Inter-population

In [ ]:
dfip = slib.htest_inter_population(data, numfields, typefields, root_table, force=False)
print(dfip.shape)
dfip.head(3)

In [ ]:
dfip2 = dfip[dfip.stat_significant == True]
dfip2.head(3)

In [ ]:
dfip2.measure.unique()

### Preparing data for plots

In [ ]:
colorGender = ["gainsboro", 'limegreen' ]  # 'ivory'
colorGender = ['lightpink', 'lightseagreen' ]
colorGender = ['lightcoral', 'lightskyblue' ]
colorGender = ['palevioletred', 'lightcyan' ]
colorGender = ['palevioletred', 'lightskyblue' ]
colorGender = ['lightcoral', 'lightsteelblue' ]

template = 'plotly_white'
template = "plotly_dark"
template = "ggplot2"
template = "seaborn"
template = "plotly_white"
template = 'plotly'
template = 'plotly_white'

In [ ]:
pops = data["ecopop"].unique().tolist()
popColors = ["green", "saddlebrown", "orange", "blue", "red", "lightblue", "silver","darkgoldenrod", "yellow"]
print(len(pops), len(popColors))
dicPopColors = {pops[i]: popColors[i]  for i in range(len(pops))}
dicPopColors

In [ ]:
dfSpecPopList = data[ ['species', 'ecopop'] ].drop_duplicates()

In [ ]:
markers = ['circle', 'circle-open', 'square', 'square-open', 'diamond', 'diamond-open', 'cross', 'x']

def species_marker(species):
    if species == specList[0]: return "circle"
    if species == specList[1]: return "square"
    if species == specList[2]: return "diamond"
    return "x"
    
def whichSpeciesMarkers(speciesSeries):
    return [species_marker(species) for species in speciesSeries]

### 3D Scatter plots (float fields)

In [ ]:
floatfields

In [ ]:
# symbols: https://plotly.com/python/reference/#scatter
from plotly.graph_objs import *

varx = 'HL'
vary = 'TaL'
varz = 'elevation'
variables = [varx, vary, varz]
units     = ['mm', 'mm', 'm']

ngender = 1
if ngender == 0:
    gender = "females"
elif ngender == 1:
    gender = 'males'
else:
    gender = 'both genders'
    
title = "Scatterplot %s x %s x %s for %s"%(varx, vary, varz, gender)

# fig = go.Figure()
traces = []

# https://stackoverflow.com/questions/47539539/separate-symbol-and-color-in-plotly-legend
n = dfSpecPopList.shape[0]

for i in range(n):
    pop     = dfSpecPopList.iloc[i].ecopop
    species = dfSpecPopList.iloc[i].species
    
    mark = species_marker(species)
    
    if ngender == 0 or ngender == 1:
        dfi = data.loc[(data.ecopop == pop) & (data.species == species) & (data.gender == ngender)]
    else:
        dfi = data.loc[(data.ecopop == pop) & (data.species == species) ]

    for igender in range(2):  
        dfi2 = dfi[dfi.gender == igender]
        if dfi2.shape[0] == 0: continue
        
        x = dfi2[varx]
        y = dfi2[vary]
        z = dfi2[varz]

        text =  ["%.1f oC and %.1f m<br>%s<br>%s"%(dfi.iloc[i].temperature, dfi.iloc[i].elevation, species, pop) 
                 for i in range(dfi.shape[0])]


        traces.append(go.Scatter3d(x=x, y=y, z=z, name = species,text=text, 
                                 mode='markers',
                                 marker={'size':8, 'color': colorGender[igender], 'symbol': mark} ))
        traces.append(go.Scatter3d(x=x, y=y, z=z, name = pop,text=text, 
                                 mode='markers',
                                 marker={'size':4, 'color':dicPopColors[pop], 'symbol': "circle"} ))


dummy_trace = go.Scatter(
    x=[None], y=[None],
    name='<b>Population</b>',
    # set opacity = 0
    line={'color': 'rgba(0, 0, 0, 0)'}
)

fig = go.Figure([dummy_trace] + traces)

fig.update_layout(
    autosize=False,
    width=1400,
    height=800,
    template=template,
    margin=dict( l=20, r=20, b=100, t=100, pad=4),
    title = title,
    font=dict(
        family="Arial, monospace",
        size=18,
        color="#7f7f7f",
    ),
    scene=Scene(
        xaxis=XAxis(title="%s (%s)"%(variables[0], units[0])),
        yaxis=YAxis(title="%s (%s)"%(variables[1], units[2])),
        zaxis=ZAxis(title="%s (%s)"%(variables[2], units[2])) ),
    legend= {'itemsizing': 'constant'},
    paper_bgcolor="whitesmoke",
    plot_bgcolor= "whitesmoke", # lightgrey ivory gainsboro whitesmoke lightsteelblue 'lightcyan' 'azure', white, lightgrey, snow ivory beige powderblue
    showlegend = True
)

filename = os.path.join(root_html, title_replace(title) + ".html")
fig.write_html(filename)

fig.show();

### 3D scatterplot - scales

In [ ]:
data.head(2)

In [ ]:
# symbols: https://plotly.com/python/reference/#scatter
from plotly.graph_objs import *

varx = 'RDS'
vary = 'VS'
varz = 'elevation'
variables = [varx, vary, varz]
units     = ['count', 'count', 'm']

ngender = 2
if ngender == 0:
    gender = "females"
elif ngender == 1:
    gender = 'males'
else:
    gender = 'both genders'
    
title = "Scatterplot %s x %s x %s for %s"%(varx, vary, varz, gender)

# fig = go.Figure()
traces = []

# https://stackoverflow.com/questions/47539539/separate-symbol-and-color-in-plotly-legend
n = dfSpecPopList.shape[0]

for i in range(n):
    pop     = dfSpecPopList.iloc[i].ecopop
    species = dfSpecPopList.iloc[i].species
    
    mark = species_marker(species)
    
    if ngender == 0 or ngender == 1:
        dfi = data.loc[(data.ecopop == pop) & (data.species == species) & (data.gender == ngender)]
    else:
        dfi = data.loc[(data.ecopop == pop) & (data.species == species) ]

    for igender in range(2):  
        dfi2 = dfi[dfi.gender == igender]
        if dfi2.shape[0] == 0: continue
        
        x = dfi2[varx]
        y = dfi2[vary]
        z = dfi2[varz]

        text =  ["%.1f oC and %.1f m<br>%s<br>%s"%(dfi.iloc[i].temperature, dfi.iloc[i].elevation, species, pop) 
                 for i in range(dfi.shape[0])]


        traces.append(go.Scatter3d(x=x, y=y, z=z, name = species,text=text, 
                                 mode='markers',
                                 marker={'size':8, 'color': colorGender[igender], 'symbol': mark} ))
        traces.append(go.Scatter3d(x=x, y=y, z=z, name = pop,text=text, 
                                 mode='markers',
                                 marker={'size':4, 'color':dicPopColors[pop], 'symbol': "circle"} ))


dummy_trace = go.Scatter(
    x=[None], y=[None],
    name='<b>Population</b>',
    # set opacity = 0
    line={'color': 'rgba(0, 0, 0, 0)'}
)

fig = go.Figure([dummy_trace] + traces)

fig.update_layout(
    autosize=False,
    width=1400,
    height=800,
    template=template,
    margin=dict( l=20, r=20, b=100, t=100, pad=4),
    title = title,
    font=dict(
        family="Arial, monospace",
        size=18,
        color="#7f7f7f",
    ),
    scene=Scene(
        xaxis=XAxis(title="%s (%s)"%(variables[0], units[0])),
        yaxis=YAxis(title="%s (%s)"%(variables[1], units[2])),
        zaxis=ZAxis(title="%s (%s)"%(variables[2], units[2])) ),
    legend= {'itemsizing': 'constant'},
    paper_bgcolor="whitesmoke",
    plot_bgcolor= "whitesmoke", # lightgrey ivory gainsboro whitesmoke lightsteelblue 'lightcyan' 'azure', white, lightgrey, snow ivory beige powderblue
    showlegend = True
)

filename = os.path.join(root_html, title_replace(title) + ".html")
fig.write_html(filename)

fig.show();

### Boxplots

In [ ]:
vars = ['RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS', 'TL', 'HL', 'TaL', 'temperature', 'elevation']
names = ['Rows dorsal scales', 'Ventral scale', 'SubTail scale', 'supra-right lips', 'supra-left lips', 
         'infra-right lips', 'infra-left lips', 'total length', 'head length', 'tail length', 'temperature', 'elevation']
paramtypes = ['int', 'int', 'int', 'int', 'int', 'int', 'int', 'float', 'float', 'float', 'float', 'float']
deltas = [1,1,1,1,1,1,1,1,1,1,1,100]
units = ['count', 'count', 'count', 'count', 'count', 'count', 'count', 'cm', 'cm', 'cm', '°C', 'm']
len(vars)

In [ ]:
br = '<br>'; L = len(specList); M = len(popList)
discretes = [0, 1]

for iloop in discretes:
    var = vars[iloop]; varname = names[iloop]; paramtype = paramtypes[iloop]; 
    unit = units[iloop]; delta = deltas[iloop]
    
    if var != varname:
        title = "Porthidium var='%s' (%s) per species - %s - both genders"%(var, unit, varname)
    else:
        title = "Porthidium var='%s' (%s) per species - both genders"%(var, unit)

       
    dfa = data[['species', var]].copy()
    # https://plotly.github.io/plotly.py-docs/generated/plotly.express.box.html
    fig = px.box(dfa, x="species", y=var, color="species", notched=False, 
                 points = 'all', color_discrete_sequence = ['green', 'turquoise', 'palevioletred'])


    fig.update_layout(
        title = title,
        autosize=False,
        width=900, height=600,
        yaxis_title = "%s (%s)"%(var, unit),
        margin=dict(l=50, r=50, b=100, t=100, pad=4),
        showlegend = False,
        paper_bgcolor="white",
        font=dict(
            family="Arial",
            size=18,
            color="#7f7f7f"
        ),
    )
    
    fig.show();
    filehtml = os.path.join(root_html, title_replace(title) + ".html")
    fig.write_html(filehtml)

In [ ]:
# https://plotly.com/python/subplots/

genders = ['female', 'male']
br = '<br>'; L = len(specList); M = len(popList)
colors = ["Tomato", "MediumBlue"]
discretes = [0, 1]

for iloop in discretes:
    var = vars[iloop]; varname = names[iloop]; paramtype = paramtypes[iloop]; 
    unit = units[iloop]; delta = deltas[iloop]
    
    if var != varname:
        title = "Porthidium var='%s' (%s) per species - %s - females x males"%(var, unit, varname)
    else:
        title = "Porthidium var='%s' (%s) per species - females x males"%(var, unit)

       
    dfa = data[['species',"gender", var]].copy()
    # https://plotly.github.io/plotly.py-docs/generated/plotly.express.box.html
    fig = px.box(dfa, x="species", y=var, color="gender", notched=False, 
                 points = 'all', color_discrete_sequence = colors)


    fig.update_layout(
        title = title,
        autosize=False,
        width=1200, height=800,
        yaxis_title = "%s (%s)"%(var, unit),
        margin=dict(l=50, r=50, b=100, t=100, pad=4),
        paper_bgcolor="white",
        showlegend = True,
        font=dict(
            family="Arial",
            size=18,
            color="#7f7f7f"
        ),
    )
    
    fig.show();
    filehtml = os.path.join(root_html, title_replace(title) + ".html")
    fig.write_html(filehtml)

In [ ]:
genders = ['female', 'male']
br = '<br>'
colors = ["Tomato", "MediumBlue"]
discretes = [0, 1]

for iloop in discretes:
    var = vars[iloop]; varname = names[iloop]; paramtype = paramtypes[iloop]; 
    unit = units[iloop]; delta = deltas[iloop]
    
    if var != varname:
        title = "Porthidium var='%s' (%s) per species-ecopop - %s - females x males"%(var, unit, varname)
    else:
        title = "Porthidium var='%s' (%s) per species-ecopop - females x males"%(var, unit)

       
    dfa = data[['species', 'ecopop', "gender", var]].copy()
    dfa["species-ecopop"] = [dfa.iloc[i].species + '-' + dfa.iloc[i].ecopop for i in range(dfa.shape[0])]

    # https://plotly.github.io/plotly.py-docs/generated/plotly.express.box.html
    fig = px.box(dfa, x="species-ecopop", y=var, color="gender", notched=False, 
                 points = 'all', color_discrete_sequence = colors)


    fig.update_layout(
        title = title,
        autosize=False,
        width=1200, height=800,
        yaxis_title = "%s (%s)"%(var, unit),
        margin=dict(l=50, r=50, b=100, t=100, pad=4),
        paper_bgcolor="white",
        showlegend = True,
        font=dict(
            family="Arial",
            size=18,
            color="#7f7f7f"
        ),
    )
    
    fig.show();
    filehtml = os.path.join(root_html, title_replace(title) + ".html")
    fig.write_html(filehtml)

### Float vars

In [ ]:
genders = ['female', 'male']
br = '<br>'; L = len(specList); M = len(popList)
colors = ["Tomato", "MediumBlue"]
floats = [7,8,9,10,11]

for iloop in floats:
    var = vars[iloop]; varname = names[iloop]; paramtype = paramtypes[iloop]; 
    unit = units[iloop]; delta = deltas[iloop]
    
    if var != varname:
        title = "Porthidium var='%s' (%s) per ecopop-species - %s - females x males"%(var, unit, varname)
    else:
        title = "Porthidium var='%s' (%s) per ecopop-species - females x males"%(var, unit)

       
    dfa = data[['species', 'ecopop', "gender", var]].copy()
    dfa["species-ecopop"] = [dfa.iloc[i].species + '-' + dfa.iloc[i].ecopop for i in range(dfa.shape[0])]

    # https://plotly.github.io/plotly.py-docs/generated/plotly.express.box.html
    fig = px.box(dfa, x="species-ecopop", y=var, color="gender", notched=False, 
                 points = 'all', color_discrete_sequence = colors)


    fig.update_layout(
        title = title,
        autosize=False,
        width=1200, height=800,
        yaxis_title = "%s (%s)"%(var, unit),
        margin=dict(l=50, r=50, b=100, t=100, pad=4),
        paper_bgcolor="white",
        font=dict(
            family="Arial",
            size=18,
            color="#7f7f7f"
        ),
    )
    
    fig.show();
    filehtml = os.path.join(root_html, title_replace(title) + ".html")
    fig.write_html(filehtml)

### Tukey test plots

In [ ]:
from statsmodels.stats.multicomp import MultiComparison
import matplotlib.gridspec as gridspec

In [ ]:
print(floatfields)
sufix_gender

In [ ]:
fig2 = plt.figure(figsize=(16,40), dpi=300, constrained_layout=False)
spec2 = gridspec.GridSpec(ncols=1, nrows=4, figure=fig2)
axes = []
axes.append(fig2.add_subplot(spec2[0]))
axes.append(fig2.add_subplot(spec2[1]))
axes.append(fig2.add_subplot(spec2[2]))
axes.append(fig2.add_subplot(spec2[3]))

units = ["cm", "cm", "oC", "m"]

numfig = 0
for line in range(4):
    var = floatfields[numfig]
    unit = units[numfig]
    numfig += 1
    title = "%s for %s"%(var, sufix_gender)
    ax = axes[line]

    model = ols('%s ~ ecopop'%(var), data=dfa).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    print(var, "ANOVA")
    print(aov_table)
    print("")

    cardata = MultiComparison(dfa[var], dfa.ecopop)
    results = cardata.tukeyhsd()
    
    title  = "Species '%s' '%s' - Tukey test: multiple comparisons between all pairs"%(species, var)
    xlabel = "%s (%s)"%(var,unit)
    
    results.plot_simultaneous(ax=ax, xlabel=xlabel)
    ax.set_title(title)
    
    print("") 
    
plt.subplots_adjust(left=0.10, right=0.95, bottom=0.05, top=0.95, hspace=2) 

## Tests and Development

### Inter species-population

In [ ]:
dfSpecPopList = data[ ['species', 'ecopop'] ].drop_duplicates()
dic = collections.OrderedDict()
count = 0

n = dfSpecPopList.shape[0]

for i in range(0,n-1):
    for k in range(i+1, n):
        df1 = dfSpecPopList.iloc[i]
        df2 = dfSpecPopList.iloc[k]
        
        species1 = df1.species
        ecopop1  = df1.ecopop
        prompt1  = species1 + '|'+ ecopop1
        
        species2 = df2.species
        ecopop2  = df2.ecopop
        prompt2  = species2 + '|'+ ecopop2

        dfv1 = data[ (data.species == species1) & (data.ecopop == ecopop1)]
        dfv2 = data[ (data.species == species2) & (data.ecopop == ecopop2)]
        
        for j in range(len(numfields)):
            measure = numfields[j]
            type    = 'discrete' if typefields[j] == 'D' else 'continuous'

            for gender in range(2):
                count += 1
                dic[count] = collections.OrderedDict()
                dic2 = dic[count]
                
                df1 = dfv1[ (dfv1.gender == gender)]
                df2 = dfv2[ (dfv2.gender == gender)]
                
                if d1.shape[0] < 2:
                    print("No data for %s, %s"%(species1, ecopop2))
                    continue                
                if d2.shape[0] < 2:
                    print("No data for %s, %s"%(species2, ecopop2))
                    continue    
                    
                    
                if type == 'discrete':
                    ret, typetest, dfgender, ns, add5, dof, stat, pvalue, stri_stat = chisquare_2series(df1, df2, measure)
                    if ret is None:
                        print("No data for gender or error for gender %d"%(gender))
                        continue
                        
                    dic2['stat_test'] = typetest
                    dic2['table'] = dfgender
                    dic2['ns'] = ns
                    dic2['add5'] = add5
                    dic2['dof'] = dof
                else:
                    ret, mat_vals, ns, stat, pvalue, stri_stat = ttest_2series(df1, df2, measure)
                    dic2['stat_test'] = 'ttest'
                    dic2['table'] = mat_vals
                    dic2['ns'] = ns
                    dic2['add5'] = False
                    dic2['dof'] = np.sum(ns) - 2


                dic2['species1'] = species1
                dic2['ecopop1']  = ecopop1
                dic2['species2'] = species2
                dic2['ecopop2']  = ecopop2
                dic2['gender']   = gender
                dic2['measure']  = measure
                dic2['type'] = type
                dic2['stat_significant'] = ret
                dic2['stat'] = stat
                dic2['pvalue'] = pvalue
                dic2['stri_stat'] = stri_stat
 
dfs = pd.DataFrame.from_dict(dic).T
dfs['fdr'] = fdr(dfs.pvalue)
dfs['stat_significant'] = dfs.fdr < 0.05

dfs = dfs[ ['species1', 'ecopop1','species2', 'ecopop2', 'gender', 'measure', 'type', 
            'ns', 'add5', 'dof', 'table',
            'stat_test', 'stat_significant', 'fdr', 'pvalue', 'stri_stat', 'stat'] ]
filechis = 'stat_chisquare_between_species_ecopops.tsv'
ret = pdwritecsv(dfs, filechis, root_table)

### Inter-species

In [ ]:
specList = data.species.unique()
dic = collections.OrderedDict()
count = 0

n = len(specList)

for i in range(0,n-1):
    for k in range(i+1, n):
        species1 = specList[i]
        species2 = specList[k]

        dfv1 = data[ (data.species == species1) ]
        dfv2 = data[ (data.species == species2) ]
        
        for j in range(len(numfields)):
            measure = numfields[j]
            type    = 'discrete' if typefields[j] == 'D' else 'continuous'
            
            for gender in range(2):
                count += 1
                dic[count] = collections.OrderedDict()
                dic2 = dic[count]
                
                df1 = dfv1[ (dfv1.gender == gender)]
                df2 = dfv2[ (dfv2.gender == gender)]
                
                if df1.shape[0] < 2:
                    print("No data for %s, %s"%(species1))
                    continue                
                if df2.shape[0] < 2:
                    print("No data for %s, %s"%(species2))
                    continue    
                    
                    
                if type == 'discrete':
                    ret, typetest, dfgender, ns, add5, dof, stat, pvalue, stri_stat = chisquare_2series(df1, df2, measure)
                    if ret is None:
                        print("No data for gender or error for gender %d"%(gender))
                        continue
                        
                    dic2['stat_test'] = typetest
                    dic2['table'] = dfgender
                    dic2['ns'] = ns
                    dic2['add5'] = add5
                    dic2['dof'] = dof
                else:
                    ret, mat_vals, ns, stat, pvalue, stri_stat = ttest_2series(df1, df2, measure)
                    dic2['stat_test'] = 'ttest'
                    dic2['table'] = mat_vals
                    dic2['ns'] = ns
                    dic2['add5'] = False
                    dic2['dof'] = np.sum(ns) - 2


                dic2['species1'] = species1
                dic2['species2'] = species2
                dic2['gender']   = gender
                dic2['measure']  = measure
                dic2['type'] = type
                dic2['stat_significant'] = ret
                dic2['stat'] = stat
                dic2['pvalue'] = pvalue
                dic2['stri_stat'] = stri_stat
 
dfs = pd.DataFrame.from_dict(dic).T
dfs['fdr'] = fdr(dfs.pvalue)
dfs['stat_significant'] = dfs.fdr < 0.05

dfs = dfs[ ['species1', 'species2', 'gender', 'measure', 'type', 
            'ns', 'add5', 'dof', 'table',
            'stat_test', 'stat_significant', 'fdr', 'pvalue', 'stri_stat', 'stat'] ]
filechis = 'stat_chisquare_between_species.tsv'
ret = pdwritecsv(dfs, filechis, root_table)

In [ ]:
dfs.head()

### Inter-population

In [ ]:
ecopopList = data.ecopop.unique()
dic = collections.OrderedDict()
count = 0

n = len(ecopopList)

for i in range(0,n-1):
    for k in range(i+1, n):
        ecopop1 = ecopopList[i]
        ecopop2 = ecopopList[k]

        dfv1 = data[ (data.ecopop == ecopop1) ]
        dfv2 = data[ (data.ecopop == ecopop2) ]
        
        for j in range(len(numfields)):
            measure = numfields[j]
            type    = 'discrete' if typefields[j] == 'D' else 'continuous'
            
            for gender in range(2):
                count += 1
                dic[count] = collections.OrderedDict()
                dic2 = dic[count]
                
                df1 = dfv1[ (dfv1.gender == gender)]
                df2 = dfv2[ (dfv2.gender == gender)]
                
                if df1.shape[0] < 2:
                    print("No data for %s, %s"%(ecopop1))
                    continue                
                if df2.shape[0] < 2:
                    print("No data for %s, %s"%(ecopop2))
                    continue    
                    
                    
                if type == 'discrete':
                    ret, typetest, dfgender, ns, add5, dof, stat, pvalue, stri_stat = chisquare_2series(df1, df2, measure)
                    if ret is None:
                        print("No data for gender or error for gender %d"%(gender))
                        continue
                        
                    dic2['stat_test'] = typetest
                    dic2['table'] = dfgender
                    dic2['ns'] = ns
                    dic2['add5'] = add5
                    dic2['dof'] = dof
                else:
                    ret, mat_vals, ns, stat, pvalue, stri_stat = ttest_2series(df1, df2, measure)
                    dic2['stat_test'] = 'ttest'
                    dic2['table'] = mat_vals
                    dic2['ns'] = ns
                    dic2['add5'] = False
                    dic2['dof'] = np.sum(ns) - 2


                dic2['ecopop1'] = ecopop1
                dic2['ecopop2'] = ecopop2
                dic2['gender']   = gender
                dic2['measure']  = measure
                dic2['type'] = type
                dic2['stat_significant'] = ret
                dic2['stat'] = stat
                dic2['pvalue'] = pvalue
                dic2['stri_stat'] = stri_stat
 
dfs = pd.DataFrame.from_dict(dic).T
dfs['fdr'] = fdr(dfs.pvalue)
dfs['stat_significant'] = dfs.fdr < 0.05

dfs = dfs[ ['ecopop1', 'ecopop2', 'gender', 'measure', 'type', 
            'ns', 'add5', 'dof', 'table',
            'stat_test', 'stat_significant', 'fdr', 'pvalue', 'stri_stat', 'stat'] ]
filechis = 'stat_chisquare_between_ecopops.tsv'
ret = pdwritecsv(dfs, filechis, root_table)

### Anova

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

for var in floatfields:
    model = ols('%s ~ ecopop'%(var), data=dfa).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    print(var)
    print(aov_table)
    print("")

### Tukey test

In [ ]:
from statsmodels.stats.multicomp import MultiComparison

In [ ]:
floatfields

In [ ]:
import matplotlib.gridspec as gridspec

fig2 = plt.figure(figsize=(16,40), dpi=300, constrained_layout=False)
spec2 = gridspec.GridSpec(ncols=1, nrows=4, figure=fig2)
axes = []
axes.append(fig2.add_subplot(spec2[0]))
axes.append(fig2.add_subplot(spec2[1]))
axes.append(fig2.add_subplot(spec2[2]))
axes.append(fig2.add_subplot(spec2[3]))

units = ["cm", "cm", "oC", "m"]

numfig = 0
for line in range(4):
    var = floatfields[numfig]
    unit = units[numfig]
    numfig += 1
    title = "%s for %s"%(var, sufix_filter)
    ax = axes[line]

    model = ols('%s ~ ecopop'%(var), data=dfa).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    print(var, "ANOVA")
    print(aov_table)
    print("")

    cardata = MultiComparison(dfa[var], dfa.ecopop)
    results = cardata.tukeyhsd()
    
    title  = "Species '%s' '%s' - Tukey test: multiple comparisons between all pairs"%(species, var)
    xlabel = "%s (%s)"%(var,unit)
    
    results.plot_simultaneous(ax=ax, xlabel=xlabel)
    ax.set_title(title)
    
    print("") 
    
plt.subplots_adjust(left=0.10, right=0.95, bottom=0.05, top=0.95, hspace=2) 



In [ ]:
var = "HL"; unit='cm'
fig, axes = plt.subplots(4,1, figsize=(16,24), dpi=300)

numfig = 0
for line in range(4):
    var = floatfields[numfig]
    numfig += 1
    title = "%s for %s"%(var, sufix_filter)
    ax = axes[line]

    model = ols('%s ~ ecopop'%(var), data=dfa).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    print(var, "ANOVA")
    print(aov_table)
    print("")

    cardata = MultiComparison(dfa[var], dfa.ecopop)
    results = cardata.tukeyhsd()
    
    title  = "Species '%s' '%s' - Tukey test: multiple comparisons between all pairs"%(species, var)
    xlabel = "%s (%s)"%(var,unit)
    
    results.plot_simultaneous(ax=ax, xlabel=xlabel)
    ax.set_title(title)
    
    print("") 
    
plt.subplots_adjust(left=0.10, right=0.95, bottom=0.05, top=0.95, hspace=2)    

In [ ]:
numfig = 0
for line in range(2):
    for col in range(2):
        var = floatfields[numfig]
        numfig += 1
        title = "%s for %s"%(var, sufix_filter)
        ax = axes[line][cols]
        
        model = ols('%s ~ ecopop'%(var), data=dfa).fit()
        aov_table = sm.stats.anova_lm(model, typ=2)
        print(var, "ANOVA")
        print(aov_table)
        print("")

        cardata = MultiComparison(dfa[var], dfa.ecopop, ax=ax)
        results = cardata.tukeyhsd()
        results.plot_simultaneous(ax=ax)
        plt.title("Species '%s' - Tukey test: multiple comparisons between all pairs"%(species))
        plt.xlabel("%s (%s)"%(var,unit))
        print("")

In [ ]:
numfig = 0
for line in range(2):
    for col in range(2):
        var = floatfields[numfig]
        numfig += 1
        title = "%s for %s"%(var, sufix_filter)
        fig, ax = plt.subplot(6, 2, numfig)
        
        model = ols('%s ~ ecopop'%(var), data=dfa).fit()
        aov_table = sm.stats.anova_lm(model, typ=2)
        print(var, "ANOVA")
        print(aov_table)
        print("")

        cardata = MultiComparison(dfa[var], dfa.ecopop, ax=ax)
        results = cardata.tukeyhsd()
        results.plot_simultaneous()
        plt.title("Species '%s' - Tukey test: multiple comparisons between all pairs"%(species))
        plt.xlabel("%s (%s)"%(var,unit))
        print("")

In [ ]:
results.summary()

In [ ]:
        for i in range(n-1):
            pop1 = ecopopList[i]

            if pop1 not in dic.keys():
                dic[pop1] = {}results
                dic[pop1]['mean'] = df.loc[(df.ecopop == pop1) & (df.species == species), var].mean()
                dic[pop1]['sd']   = df.loc[(df.ecopop == pop1) & (df.species == species), var].std()
                dic[pop1]['n']    = df.loc[(df.ecopop == pop1) & (df.species == species), var].count()

            if dic[pop1]['n'] == 0:
                    continue

            for j in range(i+1, n):
                pop2 = ecopopList[j]
                if pop2 not in dic.keys():
                    dic[pop2] = {}
                    dic[pop2]['mean'] = df.loc[(df.ecopop == pop2) & (df.species == species), var].mean()
                    dic[pop2]['sd']   = df.loc[(df.ecopop == pop2) & (df.species == species), var].std()
                    dic[pop2]['n']    = df.loc[(df.ecopop == pop2) & (df.species == species), var].count()


                if dic[pop2]['n'] == 0:
                        continue

                if verbose: print('\t',pop1, pop2)

                stat, pval = ttest_ind(df[(df.ecopop == pop1) & (df.species == species)][var],
                                       df[(df.ecopop == pop2) & (df.species == species)][var])

                dfa = pd.DataFrame({'species': [species], 'variable': [var],
                       'pop1': [pop1], 'n1': [dic[pop1]['n']], 'mean1': [dic[pop1]['mean']], 'sdv1': [dic[pop1]['sd']],
                       'pop2': [pop2], 'n2': [dic[pop2]['n']], 'mean2': [dic[pop2]['mean']], 'sdv2': [dic[pop2]['sd']],
                       'pval' : [pval], 'stat' : [stat]})

                dfpop_tt = dfpop_tt.append(dfa.iloc[0], ignore_index=True)


# Data analysis

In [ ]:
myColors = ["green", "blue", "red", "darkgoldenrod", "brown", "yellow", "orange", "gray", "navy", "pink"]
# colors = myColors[1:len(pops)]

In [ ]:
speciesList
['P_nasutum', 'P_lansbergii', 'P_sp']

In [ ]:
palette = ["green", "blue", "red"]

In [ ]:
title = "%s, morpho + temperature and elevation, not sex adjusted, by species"
title.replace(" ", "_").replace("/","-").replace(",", "")

In [ ]:
df.columns

In [ ]:
popColors = ["green", "saddlebrown", "orange", "blue", "red", "lightblue", "silver","darkgoldenrod", "yellow"]

dicPopColors = {populationList[i]: popColors[i]  for i in range(len(populationList))}
dicPopColors

In [ ]:
markers = ['circle', 'circle-open', 'square', 'square-open', 'diamond', 'diamond-open', 'cross', 'x']

def species_marker(species):
    if species == speciesList[0]: return "circle"
    if species == speciesList[1]: return "square"
    if species == speciesList[2]: return "diamond"
    return "x"
    
def whichSpeciesMarkers(speciesSeries):
    return [species_marker(species) for species in speciesSeries]

In [ ]:
# symbols: https://plotly.com/python/reference/#scatter

title = "Scatterplot for %s, VS x StS (not adjusted)"%(sufix_filter)

# fig = go.Figure()
traces = []

# https://stackoverflow.com/questions/47539539/separate-symbol-and-color-in-plotly-legend
for i in range(len(populationList)):
    pop = populationList[i]
    
    for species in speciesList:
        mark = species_marker(species)

        dfi = df.loc[(df.population == pop) & (df.species == species)]
        x = dfi.VS
        y = dfi.StS
        # speciesSeries = dfi.species
        # markList = whichSpeciesMarkers(speciesSeries)

        text =  ["%.1f m - %.1f oC<br>%s<br>%s"%(dfi.iloc[i].elevation, dfi.iloc[i].temperature, species, pop) for i in range(dfi.shape[0])]

        # fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width', color='species')
        
        traces.append(go.Scatter(x=x, y=y,name = species,text=text,
                                 mode='markers',
                                 marker={'size':14, 'color':"silver", 'symbol': mark} ))
        traces.append(go.Scatter(x=x, y=y,name = pop,text=text,
                                 mode='markers',
                                 marker={'size':5, 'color':dicPopColors[pop], 'symbol': "circle"} ))

        
dummy_trace = go.Scatter(
    x=[None], y=[None],
    name='<b>Population</b>',
    # set opacity = 0
    line={'color': 'rgba(0, 0, 0, 0)'}
)

fig = go.Figure([dummy_trace] + traces)

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict( l=50, r=50, b=100, t=100, pad=4),
    paper_bgcolor="white",
    plot_bgcolor= "whitesmoke", # 'lightcyan' 'azure', whitesmoke', white, lightgrey, snow ivory beige powderblue
    title = title,
    xaxis_title="VS",
    yaxis_title="StS",
    font=dict(
        family="Arial, monospace",
        size=18,
        color="#7f7f7f"
    )
    
)

fig.show();

### 3D

In [ ]:
from plotly.graph_objs import *

# symbols: https://plotly.com/python/reference/#scatter

title = "Scatterplot for %s, VS x StS (not adjusted)"%(sufix_filter)
# fig = go.Figure()
traces = []

# https://stackoverflow.com/questions/47539539/separate-symbol-and-color-in-plotly-legend
for i in range(len(populationList)):
    pop = populationList[i]
    
    for species in speciesList:
        mark = species_marker(species)

        dfi = df.loc[(df.population == pop) & (df.species == species)]
        x = dfi.VS
        y = dfi.StS
        z = dfi.elevation
        # speciesSeries = dfi.species
        # markList = whichSpeciesMarkers(speciesSeries)

        text =  ["%.1f oC and %.1f m<br>%s<br>%s"%(dfi.iloc[i].temperature, dfi.iloc[i].elevation, species, pop) 
                 for i in range(dfi.shape[0])]

        
        traces.append(go.Scatter3d(x=x, y=y, z=z, name = species,text=text,
                                 mode='markers',
                                 marker={'size':8, 'color':"gainsboro", 'symbol': mark} ))  # lightsteelblue beige
        traces.append(go.Scatter3d(x=x, y=y, z=z, name = pop,text=text,
                                 mode='markers',
                                 marker={'size':4, 'color':dicPopColors[pop], 'symbol': "circle"} ))

        
dummy_trace = go.Scatter(
    x=[None], y=[None],
    name='<b>Population</b>',
    # set opacity = 0
    line={'color': 'rgba(0, 0, 0, 0)'}
)

fig = go.Figure([dummy_trace] + traces)

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict( l=20, r=20, b=100, t=100, pad=4),
    title = title,
    font=dict(
        family="Arial, monospace",
        size=18,
        color="#7f7f7f",
    ),
    scene=Scene(
        xaxis=XAxis(title='VS'),
        yaxis=YAxis(title='StS'),
        zaxis=ZAxis(title='altitude (m)'),
    ),
    legend= {'itemsizing': 'constant'},
    paper_bgcolor="whitesmoke",
    plot_bgcolor= "whitesmoke", # gainsboro whitesmoke lightsteelblue 'lightcyan' 'azure', whitesmoke', white, lightgrey, snow ivory beige powderblue
    showlegend = True
)

filename = rootHTML + title.replace(" ", "_").replace("/","-").replace(",", "") + ".html"
fig.write_html(filename)

fig.show();

In [ ]:
# g = sns.pairplot(df3, hue='species', height=6, aspect=1, size=2.5)
plt.figure(figsize=(16,14))
sns.set(font_scale=1.3)
title = "Scatterplot for %s, VS x StS x temperature level (not adjusted)"%(sufix_filter)

g = sns.scatterplot(x = "VS", y= "StS", style='tempLevel', hue='population', data=df, 
                    legend='full', palette=popColors, s=200)

plt.xlabel("Ventral scales (VS)")
plt.ylabel("Subtail scales (StS)")
plt.title(title)
# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

filename = root + title.replace(" ", "_").replace("/","-").replace(",", "") + ".png"
plt.savefig(filename, bbox_inches='tight')

In [ ]:
# g = sns.pairplot(df3, hue='species', height=6, aspect=1, size=2.5)
plt.figure(figsize=(16,14))
sns.set(font_scale=1.3)
title = "Scatterplot for %s, VS x StS x elevation level (not adjusted)"%(sufix_filter)

g = sns.scatterplot(x = "VS", y= "StS", style='elevLevel', hue='population', data=df, 
                    legend='full', palette=popColors, s=200)

plt.xlabel("Ventral scales (VS)")
plt.ylabel("Subtail scales (StS)")
plt.title(title)
# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

filename = root + title.replace(" ", "_").replace("/","-").replace(",", "") + ".png"
plt.savefig(filename, bbox_inches='tight')

### Random Forest 

https://stackoverflow.com/questions/8961586/do-i-need-to-normalize-or-scale-data-for-randomforest-r-package

do I need to normalize (or scale) data for randomForest?

Hong Ooi:  
No, scaling is not necessary for random forests.  

The nature of RF is such that convergence and numerical precision issues, which can sometimes trip up the algorithms used in logistic and linear regression, as well as neural networks, aren't so important. Because of this, you don't need to transform variables to a common scale like you might with a NN.  

You're don't get any analogue of a regression coefficient, which measures the relationship between each predictor variable and the response. Because of this, you also don't need to consider how to interpret such coefficients which is something that is affected by variable measurement scales.


In [ ]:
factors = pd.factorize(df["population"])[0]
set(factors)

In [ ]:
factors

In [ ]:
n = len(factors)
ntrain = int(n*.6)
ntest  = int(n*.4)
n, ntrain, ntest

In [ ]:
df.index.tolist()[:8]

In [ ]:
df[ df.population == populationList[1] ].index.tolist()[:8]

In [ ]:
all = np.arange(162,169+1)
print(all)
np.random.shuffle(all)
print(all)

### Random Forest Population

In [ ]:
df2 = df[ ["species", 'population'] + numfields]
df2.head()

In [ ]:
dic = {}
trains = []; tests = []

for pop in set(populationList):
    m = np.array(df2[ df2.population == pop ].index.tolist())
    np.random.shuffle(m)
    n = len(m)
    ntrain = int(n*.6)
   
    train = m[:ntrain]
    test  = m[ntrain:]
    # print(pop, train, test)
    dic[pop] = {}
    dic[pop]["train"] = train
    dic[pop]["test"] = test
    
    trains += list(train)
    tests += list(test)
    
trains.sort()
tests.sort()

print(trains)
print(tests)
print(len(trains)+len(tests))
df2.shape

In [ ]:
df2.iloc[tests]

In [ ]:
start_at = 2
# df, df2, only one sex
dfTrain = df2.iloc[trains]
X_train = dfTrain[ dfTrain.columns[start_at:] ]
y_train = pd.factorize(dfTrain["population"])[0]

dfTest = df2.iloc[tests]
X_test = dfTest[ dfTest.columns[start_at:] ]
y_test = pd.factorize(dfTest["population"])[0]

print("Train (pop):", X_train.shape[0], len(y_train), y_train[:30])
print("")
print("Test  (pop):", X_Test.shape[0], len(y_test), y_test[:30])
X_train.head()

In [ ]:
print(y_test)
print(len(y_test))
print(X_test.shape[0])
X_test.head()

In [ ]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
# https://chrisalbon.com/machine_learning/trees_and_forests/random_forest_classifier_example/

model = RandomForestClassifier(random_state=0, n_estimators=5000, n_jobs=6)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
f = model.fit(X_train, y_train)

In [ ]:
prediction = model.predict(X_test)
prediction

In [ ]:
np.array([True if x == 0 else False for x in (prediction - y_test)]).sum()

In [ ]:
nTest = X_test.shape[0]
np.array([True if x == 0 else False for x in (prediction - y_test)]).sum() / nTest

In [ ]:
# View the predicted probabilities of the first 10 observations
model.predict_proba(X_test)[0:10]

In [ ]:
# Create actual english names for the plants for each predicted plant class
dfpred = df.iloc[prediction]
dfpred.index = list(range(dfpred.shape[0]))
dfpred.head(3)

In [ ]:
dfTest.index = list(range(dfTest.shape[0]))
dfTest.head(3)

In [ ]:
dfTest.shape[0], dfpred.shape[0]

In [ ]:
pd.crosstab(dfTest.population, dfTest.population.unique()[prediction], rownames=['actual population'], colnames=['predicted poputlation'], margins=True)

In [ ]:
feature_names = X_test.columns
feature_names

In [ ]:
str(model)

In [ ]:
model.estimators_[100]

### Lasso

http://www.science.smith.edu/~jcrouser/SDS293/labs/lab10-py.html

We saw that ridge regression with a wise choice of alpha can outperform least squares as well as the null model on the our data set. We now ask whether the lasso can yield either a more accurate or a more interpretable model than ridge regression. In order to fit a lasso model, we'll use the Lasso() function; however, this time we'll need to include the argument max_iter = 10000. Other than that change, we proceed just as we did in fitting a ridge model

In [ ]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error

In [ ]:
alphas = 10**np.linspace(1,-2,100)*0.5
alphas

In [ ]:
dfa = df[["species"] + numfields]
dfa.head()

In [ ]:
(dfa[numfields] == 0).any(axis=0)

In [ ]:
df3 = df2


    
df3.head()

In [ ]:
# Split data into training and test sets
# X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

dic = {}
trains = []; tests = []

for species in set(speciesList):
    m = np.array(df3[ df3.species == species ].index.tolist())
    np.random.shuffle(m)
    n = len(m)
    ntrain = int(n*.6)
   
    train = m[:ntrain]
    test  = m[ntrain:]
    # print(pop, train, test)
    dic[pop] = {}
    dic[pop]["train"] = train
    dic[pop]["test"] = test
    
    trains += list(train)
    tests += list(test)


start_at = 2
# df, df2, only one sex
dfTrain = df2.iloc[trains]
X_train = dfTrain[ dfTrain.columns[start_at:] ]
y_train = pd.factorize(dfTrain["species"])[0]

dfTest = df2.iloc[tests]
X_test = dfTest[ dfTest.columns[start_at:] ]
y_test = pd.factorize(dfTest["species"])[0]

print("Train (pop):", X_train.shape[0], len(y_train), y_train[:30])
print("")
print("Test  (pop):", X_Test.shape[0], len(y_test), y_test[:30])
X_train.head()

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(X_train), y_train)
    coefs.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

Notice that in the coefficient plot that depending on the choice of tuning parameter, some of the coefficients are exactly equal to zero. We now perform 10-fold cross-validation to choose the best alpha, refit the model, and compute the associated test error:

In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
mean_squared_error(y_test, lasso.predict(X_test))

In [ ]:
X_train.columns

This is substantially lower than the test set MSE of the null model and of least squares, and only a little worse than the test MSE of ridge regression with alpha chosen by cross-validation.  

However, the lasso has a substantial advantage over ridge regression in that the resulting coefficient estimates are sparse. Here we see that 13 of the 19 coefficient estimates are exactly zero:

In [ ]:
# Some of the coefficients are now reduced to exactly zero.
pd.Series(lasso.coef_, index=X_train.columns)

In [ ]:
# ser = pd.Series(lasso.coef_, index=numfields)
dic = {X_train.columns[i]: [lasso.coef_[i]] for i in range(len(X_train.columns))}
dfl = pd.DataFrame(dic).T
dfl.reset_index(level=0, inplace=True)
dfl.columns = ["param", "coef"]
dfl

In [ ]:
dfl = dfl[dfl.coef != 0]
dfl = dfl.sort_values("coef", ascending=False)
dfl

### Lasso another example

https://www.kirenz.com/post/2019-08-12-python-lasso-regression-auto

In [ ]:
reg = Lasso(alpha=0.001, max_iter=1000000, fit_intercept=False)
reg.fit(X_train, y_train)

In [ ]:
print('Lasso Regression: R^2 score on training set', reg.score(X_train, y_train)*100)
print('Lasso Regression: R^2 score on test set', reg.score(X_test, y_test)*100)

#### Lasso with different lambdas

Apply the Lasso regression on the training set with the following λ parameters: (0.001, 0.01, 0.1, 0.5, 1, 2, 10). Evaluate the R^2 score for all the models you obtain on both the train and test sets.

In [ ]:
lambdas = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2]
l_num = len(lambdas)
pred_num = len(X_train.columns)
l_num, pred_num

In [ ]:
# prepare data for enumerate
coeff_a = np.zeros((l_num, pred_num))
train_r_squared = np.zeros(l_num)
test_r_squared = np.zeros(l_num)

In [ ]:
X_train.columns

In [ ]:
X_test.columns

In [ ]:
y_test

In [ ]:
# enumerate through lambdas with index and i
for ind, i in enumerate(lambdas):    
    reg = Lasso(alpha = i, max_iter=100000, fit_intercept=True)
    reg.fit(X_train, y_train)

    coeff_a[ind,:] = reg.coef_
    train_r_squared[ind] = reg.score(X_train, y_train)
    test_r_squared[ind] = reg.score(X_test, y_test)

In [ ]:
# Plotting
plt.figure(figsize=(18, 8))
plt.plot(train_r_squared, 'bo-', label=r'$R^2$ Training set', color="darkblue", alpha=0.6, linewidth=3)
plt.plot(test_r_squared, 'bo-', label=r'$R^2$ Test set', color="darkred", alpha=0.6, linewidth=3)
plt.xlabel('Lamda index'); plt.ylabel(r'$R^2$')
plt.xlim(0, 6)
plt.title(r'Evaluate lasso regression with lamdas: 0 = 0.001, 1= 0.01, 2 = 0.1, 3 = 0.5, 4= 1, 5= 2, 6 = 10')
plt.legend(loc='best')
plt.grid()

### Random forest tree

In [ ]:
from sklearn.tree import export_graphviz
from subprocess import call

In [ ]:
estimator = model.estimators_[4]
estimator

In [ ]:
rootpng = '../data2020/png/'
name = "rf_colombia_prothidium_population"
filedot = os.path.join(root+"png/", name+'.dot')

filedot = name+'.dot'
filepng = filedot.replace(".dot", ".png")

filedot, filepng

In [ ]:
# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_names,
                class_names = dfTest.population.unique(),
                rounded = True, proportion = False, 
                precision = 2, filled = True)
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
estimator = model.estimators_[4]

print(filedot, filepng)

# Export as dot file
export_graphviz(estimator, out_file=filedot, 
                feature_names = feature_names,
                class_names = dfTest.population.unique(),
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
# call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])
subprocess.call(['dot', '-Tpng', filedot, '-o', filepng, '-Gdpi=200'])

# Display in jupyter notebook
from IPython.display import Image

Image(filename = filename)


### Random Forest Species

In [ ]:
dic = {}
trains = []; tests = []

for species in set(speciesList):
    m = np.array(df[ df.species == species ].index.tolist())
    np.random.shuffle(m)
    n = len(m)
    ntrain = int(n*.6)
   
    train = m[:ntrain]
    test  = m[ntrain:]
    # print(pop, train, test)
    dic[pop] = {}
    dic[pop]["train"] = train
    dic[pop]["test"] = test
    
    trains += list(train)
    tests += list(test)
    
trains.sort()
tests.sort()

print(trains)
print(tests)
print(len(trains)+len(tests))
df.shape

In [ ]:
start_at = 2
# df, df2, only one sex
dfTrain = df2.iloc[trains]
X_train = dfTrain[ dfTrain.columns[start_at:] ]
y_train = pd.factorize(dfTrain["species"])[0]

dfTest = df2.iloc[tests]
X_test = dfTest[ dfTest.columns[start_at:] ]
y_test = pd.factorize(dfTest["species"])[0]

print("Train (pop):", X_train.shape[0], len(y_train), y_train[:30])
print("")
print("Test  (pop):", X_Test.shape[0], len(y_test), y_test[:30])
X_train.head()

In [ ]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
# https://chrisalbon.com/machine_learning/trees_and_forests/random_forest_classifier_example/

modelspec = RandomForestClassifier(random_state=0, n_estimators=12000, n_jobs=6)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
f = modelspec.fit(X_train, y_train)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
prediction = modelspec.predict(X_test)
len(prediction), prediction

In [ ]:
pd.crosstab(dfTest.species, dfTest.species.unique()[prediction], rownames=['actual population'], colnames=['predicted poputlation'], margins=True)

In [ ]:
nTest = X_test.shape[0]
1-np.array([True if x == 0 else False for x in (prediction - y_test)]).sum() / nTest

In [ ]:
feature_names = X_test.columns
feature_names

In [ ]:
model.estimators_[1000]

In [ ]:
estimator = modelspec.estimators_[4]

name = "rf_colombia_prothidium_by_species"
filedot = os.path.join(root+"png/", name+'.dot')

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file=filedot, 
                feature_names = feature_names,
                class_names = dfTest.species.unique(),
                rounded = True, proportion = False, 
                precision = 2, filled = True)

filename = os.path.join(root+"png/", name+'.png')

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', name+'.dot', '-o',filename, '-Gdpi=200'])

# Display in jupyter notebook
from IPython.display import Image

print(filename)
Image(filename = filename)

### PCA - principal component analysis - Vent Subc LT

In [ ]:
df.columns

In [ ]:
df2 = df[fieldsNorm]

In [ ]:
colors = [dicColors[pop] for pop in df2.population]
np.array(colors)[1:50]

In [ ]:
fields2 = ['sex', 'RDS', 'VS', 'StS', 'SRS', 'SLS', 'IRS', 'ILS', 'GsS', 'PvS', 'LevR', 'DP']

In [ ]:
df2['sex'] = [1 if x=='male' else 2 for x in df2['sex'] ]

In [ ]:
df2[fields2].head()

In [ ]:
pca = PCA()
fit = pca.fit_transform(df2[fields2])

In [ ]:
%matplotlib inline
import matplotlib.patches as mpatches
# fig = plt.figure(figsize=(10,10))

fig, ax = plt.subplots(figsize=(15,10))
plt.scatter(fit[:,0], fit[:,1], s=100, c=colors, alpha=1)

# ax.legend(lines[:2], ['line A', 'line B'], loc='upper right', frameon=False)
# for pop in dicColors.keys():
patches = [ mpatches.Patch(color=dicColors[pop], label=pop) for pop in dicColors.keys() ]

plt.ylabel("PC2")
plt.xlabel("PC1")
plt.title("PCA is a bad clusterization model - 3 vars reache less then 70% exp.var.")
plt.legend(handles=patches, loc="lower right")

In [ ]:
pca.explained_variance_ratio_

In [ ]:
soma = 0; vals = []
for val in pca.explained_variance_ratio_:
    soma += val
    vals.append(soma)

In [ ]:
soma=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3)*100)

In [ ]:
plt.plot(vals)
plt.ylabel("Explained variance (%)")
plt.xlabel("Eigenvectors")
plt.title("PCA is a bad clusterization model with 3 vars it reaches ~.7")

In [ ]:
# pca.components_

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

fig = plt.figure(figsize=(12,10))
ax = Axes3D(fig)

# PCA 3 first components
varx = fit[:,0]
vary = fit[:,1]
varz = fit[:,2]

ax.scatter( varx, vary, varz, c=colors)

ax.set_xlabel("pc1")
ax.set_ylabel("pc2")
ax.set_zlabel("pc3")

# label=df2.Population
# ax.legend(loc="upper right", title="regions")

In [ ]:
pops

In [ ]:
def chisquare_gender01_female_male(df2, measure, field):
    valsf = df2[df2[field] == 0].groupby(measure).count().iloc[:,0:1]
    valsf['class'] = valsf.index
    valsf.columns = ['count', 'class']
    valsf = valsf[['class', 'count']].T

    valsm = df2[df2.gender == 1].groupby(measure).count().iloc[:,0:1]
    valsm['class'] = valsm.index
    valsm.columns = ['count', 'class']
    valsm = valsm[['class', 'count']].T
    
    cols = np.unique(list(valsf.columns) + list(valsm.columns))
    cols = np.sort(cols)
    dic = {}
    for col in cols:
        dic[col] = []

        try:
            dic[col].append(valsf[col][1])
        except:
            dic[col].append(0)

        try:
            dic[col].append(valsm[col][1])
        except:
            dic[col].append(0)   

    dfgender = pd.DataFrame.from_dict(dic)
    dfgender.index = ['female', 'male']
    print(dfgender)   
    
    stat, pvalue = scipy.stats.chisquare(np.array([ dfgender.iloc[0].to_list(), dfgender.iloc[1].to_list()]), axis=None)
    return stat, pvalue


In [ ]:

def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    mat = []
    groups = results.groupsunique
    for i in range(len(groups)-1):
        for j in range(i+1, len(groups)):
            mat.append((groups[i], groups[j]))

    mat = np.array(mat)
    group1 = np.array(mat)[:, 0]
    group2 = np.array(mat)[:, 1]

    pvals = results.pvalues
    meandiffs = results.meandiffs
    conf_lower = np.array(results.confint)[:, 0]
    conf_higher = np.array(results.confint)[:, 1]
    reject = results.reject

    results_df = pd.DataFrame({"group1": group1,
                               "group2": group2,
                               "pvalue_adj":pvals,
                               "meandiffs":meandiffs,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher,
                               "reject": reject })

    #Reordering...
    results_df = results_df[["group1", "group2", "meandiffs","pvalue_adj","conf_lower","conf_higher"]]
    return results_df
    